In [ ]:

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !cp "C:\Usersuper-res-large.zip"
# !unzip -q  dataset.zip 

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Model
#from keras.applications.vgg19 import VGG19
from  tensorflow.keras.applications import VGG19
from PIL import Image
import tqdm

In [ ]:
db_dir = "./input/srgandataset/dataset-lg"
data=tfds.load('tf_flowers')

In [ ]:
train_data=data['train'].skip(500)
test_data=data['train'].take(500)


In [ ]:
tqdm.tqdm(train_data)

In [ ]:
@tf.function
def build_data(data):
  cropped=tf.dtypes.cast(tf.image.random_crop(data['image'] / 255,(128,128,3)),tf.float32)

  lr=tf.image.resize(cropped,(32,32))
  #lr=tf.image.resize(lr,(128,128),method=tf.image.ResizeMethod.BICUBIC)
  return (lr,cropped)

In [ ]:
for x in train_data.take(1):
  plt.imshow(x['image'])
  plt.show()

In [ ]:
def bicubic_interpolate(image,shape):
  img_resized=cv2.resize(image,shape, interpolation=cv2.INTER_CUBIC)
  return img_resized

In [ ]:
train_dataset_mapped = train_data.map(build_data,num_parallel_calls=tf.data.AUTOTUNE)
for x in train_dataset_mapped.take(1):
  plt.imshow(x[0].numpy())
  plt.show()

  print(" After applying Bicubic Interpolation \n")
  plt.imshow(bicubic_interpolate(x[0].numpy(),(128,128))) 
  plt.show()
  plt.imshow(x[1].numpy())
  plt.show()

In [ ]:
# def get_train_data():
#     X = []
#     Y = []
#     for x in os.listdir(db_dir + "/train/lr"):
#         img_x = cv2.imread(db_dir + "/train/lr/" + x)
#         X.append(img_x)
    
#     X = np.array(X) / 255
        
#     for y in os.listdir(db_dir + "/train/hr"):
#         img_y = cv2.imread(db_dir + "/train/hr/" + y)
#         Y.append(img_y)
#     Y = np.array(Y) / 255 
    
#     return X,Y




In [ ]:
for x in range(50):
    train_dataset_mapped = train_data.map(build_data,num_parallel_calls=tf.data.AUTOTUNE).batch(64)
    val_dataset_mapped = test_data.map(build_data,num_parallel_calls=tf.data.AUTOTUNE).batch(64)
X=np.array(train_dataset_mapped)
print(X)



In [ ]:

# print("train_shape ", train_lr.shape, " test_shape ",test_lr.shape  )

num_res_block = 16
# hr_shape = (x[1].shape[1], x[1].shape[2], x[1].shape[3])
# lr_shape = (x[0].shape[1], x[0].shape[2], x[0].shape[3])


Conv2D = layers.Conv2D
BatchNormalization = layers.BatchNormalization
PReLU = layers.PReLU
UpSampling2D = layers.UpSampling2D
Dense = layers.Dense
add = layers.add
LeakyReLU = layers.LeakyReLU
Input = layers.Input
Flatten = layers.Flatten


lr_ip = Input(shape=(32,32,3))
hr_ip = Input(shape=(128,128,3))

## Generator

In [ ]:
def res_block(ip):
    
    res_model = Conv2D(64, (3,3), padding = "same")(ip)
    res_model = BatchNormalization(momentum = 0.5)(res_model)
    res_model = PReLU(shared_axes = [1,2])(res_model)
    
    res_model = Conv2D(64, (3,3), padding = "same")(res_model)
    res_model = BatchNormalization(momentum = 0.5)(res_model)
    
    return add([ip,res_model])

def upscale_block(ip):
    
    up_model = Conv2D(256, (3,3), padding="same")(ip)
    up_model = UpSampling2D( size = 2 )(up_model)
    up_model = PReLU(shared_axes=[1,2])(up_model)
    
    return up_model

## Discriminator

In [ ]:


def discriminator_block(ip, filters, strides=1, bn=True):
    
    disc_model = Conv2D(filters, (3,3), strides = strides, padding="same")(ip)
    disc_model = LeakyReLU( alpha=0.2 )(disc_model)
    if bn:
        disc_model = BatchNormalization( momentum=0.8 )(disc_model)

    
    return disc_model
    

## Generator model

In [ ]:
def create_gen(gen_ip):
    layers = Conv2D(64, (9,9), padding="same")(gen_ip)
    layers = PReLU(shared_axes=[1,2])(layers)

    temp = layers

    for i in range(num_res_block):
        layers = res_block(layers)

    layers = Conv2D(64, (3,3), padding="same")(layers)
    layers = BatchNormalization(momentum=0.5)(layers)
    layers = add([layers,temp])

    layers = upscale_block(layers)
    layers = upscale_block(layers)

    op = Conv2D(3, (9,9), padding="same")(layers)

    return Model(inputs=gen_ip, outputs=op)

## Discriminator model

In [ ]:
def create_disc(disc_ip):

    df = 64
    
    d1 = discriminator_block(disc_ip, df, bn=False)
    d2 = discriminator_block(d1, df, strides=2)
    d3 = discriminator_block(d2, df*2)
    d4 = discriminator_block(d3, df*2, strides=2)
    d5 = discriminator_block(d4, df*4)
    d6 = discriminator_block(d5, df*4, strides=2)
    d7 = discriminator_block(d6, df*8)
    d8 = discriminator_block(d7, df*8, strides=2)
    
    d8_5 = Flatten()(d8)
    d9 = Dense(df*16)(d8_5)
    d10 = LeakyReLU(alpha=0.2)(d9)
    validity = Dense(1, activation='sigmoid')(d10)

    return Model(disc_ip, validity)
    


## VGG19

In [ ]:
def build_vgg():
           #  establish VGG Model , Use only the 9 Characteristics of the layer 
    vgg = VGG19(weights="imagenet",input_shape=hr_ip,include_top=False)
    return Model(vgg.input, outputs=vgg.layers[9].output)
    # vgg = 
    # vgg.outputs = [vgg.layers[9].output]

    # img = Input(weights="imagenet",input_shape=self.hr_shape,include_top=False)

    # img_features = vgg(img)

    # return Model(vgg.input, outputs=vgg.layers[9].output)



## Combined Model

In [ ]:

def create_comb(gen_model, disc_model, vgg,lr_ip,hr_ip):
    gen_img = gen_model(lr_ip)
    
    gen_features = vgg(gen_img)
    
    disc_model.trainable = False
    validity = disc_model(gen_img)
    
    return Model(inputs=[(32, 32, 3),(128, 128, 3) ], outputs=[validity, gen_features])


In [ ]:

generator = create_gen(lr_ip)
discriminator = create_disc(hr_ip)
discriminator.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

vgg = build_vgg()
vgg.trainable = False


gan_model = create_comb(generator, discriminator, vgg, lr_ip, hr_ip)
gan_model.compile(loss=["binary_crossentropy","mse"], loss_weights=[1e-3, 1], optimizer="adam")



In [ ]:
# discriminator.summary()
# generator.summary()
gan_model.summary()

In [ ]:
batch_size = 100
train_lr_batches = []
train_hr_batches = []
for it in range(int(train_hr.shape[0] / batch_size)):
    start_idx = it * batch_size
    end_idx = start_idx + batch_size
    train_hr_batches.append(train_hr[start_idx:end_idx])
    train_lr_batches.append(train_lr[start_idx:end_idx])

In [ ]:

epochs = 50
for e in range(epochs):
    
    gen_label = np.zeros((batch_size, 1))
    real_label = np.ones((batch_size,1))
    g_losses = []
    d_losses = []
    for b in range(len(train_hr_batches)):
        lr_imgs = train_lr_batches[b]
        hr_imgs = train_hr_batches[b]
        
        gen_imgs = generator.predict_on_batch(lr_imgs)
        
        discriminator.trainable = True
        d_loss_gen = discriminator.train_on_batch(gen_imgs, gen_label)
        d_loss_real = discriminator.train_on_batch(hr_imgs, real_label)
        discriminator.trainable = False
        
        d_loss = 0.5 * np.add(d_loss_gen, d_loss_real) 
        
        image_features = vgg.predict(hr_imgs)

        
        g_loss, _, _ = gan_model.train_on_batch([lr_imgs, hr_imgs], [real_label, image_features])
        
        d_losses.append(d_loss)
        g_losses.append(g_loss)
        
    g_losses = np.array(g_losses)
    d_losses = np.array(d_losses)
    
    g_loss = np.sum(g_losses, axis=0) / len(g_losses)
    d_loss = np.sum(d_losses, axis=0) / len(d_losses)
    
    print("epoch:", e+1 ,"g_loss:", g_loss, "d_loss:", d_loss)

    if (e+1) % 20 == 0:
        discriminator.save_weights("./e_"+ str(e+1) +".h5")
        generator.save_weights("./e_"+ str(e+1) +".h5")


In [ ]:
res = generator.predict_on_batch(train_lr_batches[0])
plt.imshow(res[0])
plt.show()
plt.imshow(train_lr_batches[0][0])
plt.show()
plt.imshow(train_hr_batches[0][0])
plt.show() 

In [ ]:
test_label = np.ones((len(test_lr),1))
test_imgs_features = vgg.predict(test_hr)
test_res = gan_model.evaluate([test_lr, test_hr], [test_label, test_imgs_features],10 )
print("Test loss & test Accuracy:", test_res)


In [ ]:
test_gen = generator.predict_on_batch(test_lr)
plt.imshow(test_lr[3])
plt.show()
plt.imshow(test_gen[3])
plt.show()
plt.imshow(test_hr[3])
plt.show()